In [6]:
import keras
from keras.utils.np_utils import to_categorical
from keras.layers import Dense,Dropout,Activation,LSTM
from keras.models import Model,Sequential
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [7]:

from music21 import converter, instrument,note,chord

In [8]:
import numpy as np

In [9]:
import glob

In [10]:

notes=[]
for file in glob.glob("C:/Users/user/cpc/midi_songs/*.mid"):
    midi= converter.parse(file)
    #print(midi)
    notes_to_parse=[]
    parts=instrument.partitionByInstrument(midi)
    #print(parts)
    if parts:
        notes_to_parse=parts.parts[0].recurse()
    else:
        notes_to_parse= midi.flat_notes
        
    print(notes_to_parse)
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            notes.append(str(element.pitch))
        elif isinstance( element, chord.Chord):
            notes.append('.'.join(str(n) for n in element.normalOrder))
#print(notes)

<music21.stream.iterator.RecursiveIterator for Part:Piano @:0>
<music21.stream.iterator.RecursiveIterator for Part:Piano @:0>
<music21.stream.iterator.RecursiveIterator for Part:Piano @:0>
<music21.stream.iterator.RecursiveIterator for Part:Piano @:0>
<music21.stream.iterator.RecursiveIterator for Part:Piano @:0>
<music21.stream.iterator.RecursiveIterator for Part:Piano @:0>
<music21.stream.iterator.RecursiveIterator for Part:Piano @:0>
<music21.stream.iterator.RecursiveIterator for Part:Piano @:0>
<music21.stream.iterator.RecursiveIterator for Part:Piano @:0>
<music21.stream.iterator.RecursiveIterator for Part:Piano @:0>
<music21.stream.iterator.RecursiveIterator for Part:Piano @:0>
<music21.stream.iterator.RecursiveIterator for Part:Piano @:0>
<music21.stream.iterator.RecursiveIterator for Part:Piano @:0>
<music21.stream.iterator.RecursiveIterator for Part:Piano @:0>
<music21.stream.iterator.RecursiveIterator for Part:Piano @:0>
<music21.stream.iterator.RecursiveIterator for Part:Pia

In [11]:
print(notes[:4])

['4.9', 'E2', '4.9', '4.9']


In [12]:
seq_len=100
pitchnames=sorted(set(item for item in notes))
note2int = dict((note,number) for number,note in enumerate(pitchnames))

In [13]:
net_i=[]
net_o=[]

for i in range(0,len(notes)-seq_len,1):
    seqi=notes[i:i+seq_len]
    #print(seqi)
    seqo=notes[i+seq_len]
    #print(seqi)
    #print(seqo)
    
    net_i.append([note2int[t] for t in seqi])
    
    net_o.append([note2int[seqo]])

In [14]:

n_patterns=len(net_i)
n_vocab=len(note2int)
net_i= np.reshape(net_i,(n_patterns,seq_len,1))
net_i=net_i/float(n_vocab)


In [15]:
net_o=to_categorical(net_o)
net_o

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [16]:
net_o.shape



(57077, 358)

In [21]:
def make_model():
    model = Sequential()
    model.add(LSTM(512,input_shape=(net_i.shape[1], net_i.shape[2]),return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(512, return_sequences=True))
    model.add(Dropout(0.3))
    model.add(LSTM(512, return_sequences=False))
    model.add(Dense(256))#activation='relu')
    model.add(Activation('relu'))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))#,activation='softmax')
    model.add(Activation('softmax'))
    return model
model = make_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 100, 512)          1052672   
_________________________________________________________________
dropout_3 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_4 (LSTM)                (None, 100, 512)          2099200   
_________________________________________________________________
dropout_4 (Dropout)          (None, 100, 512)          0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 512)               2099200   
_________________________________________________________________
dense_3 (Dense)              (None, 256)               131328    
_________________________________________________________________
activation_3 (Activation)    (None, 256)               0         
__________

In [22]:
filepath = "weights-improvement-74+={epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(
        filepath,
        monitor='loss',
        verbose=0,
        save_best_only=True,
        mode='min'
    )
callbacks_list = [checkpoint]


In [23]:
model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy'])

In [24]:
model.load_weights('../pianofiles/3lstm_65_acc_88.h5')

In [55]:
model.fit(net_i, net_o, epochs=1, batch_size=64,callbacks=callbacks_list)

Epoch 1/1
12544/57077 [=====>........................] - ETA: 2:55 - loss: 1.3909 - accuracy: 0.6987

KeyboardInterrupt: 

In [ ]:
#model.save('modelfinal.h5')

OSError: Unable to create file (unable to open file: name = '../piano_model/modelfinal.h5', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 242)

In [ ]:
model.save_weights("weights_epoch80_acc_82.h5")

In [25]:
start= np.random.randint(0,len(net_i)-1)
pattern= net_i[start]


In [30]:
pattern

array([0.89385475, 0.81005587, 0.99441341, 0.83519553, 0.87430168,
       0.05586592, 0.95251397, 0.84636872, 0.84636872, 0.84636872,
       0.87150838, 0.84636872, 0.83240223, 0.97486034, 0.95251397,
       0.95251397, 0.95530726, 0.82681564, 0.18994413, 0.79329609,
       0.94413408, 0.97486034, 0.86871508, 0.93854749, 0.97486034,
       0.79608939, 0.83240223, 0.94413408, 0.87430168, 0.74022346,
       0.97486034, 0.81005587, 0.79329609, 0.97486034, 0.81005587,
       0.88826816, 0.87150838, 0.79050279, 0.83519553, 0.97765363,
       0.57541899, 0.79888268, 0.97206704, 0.79329609, 0.90782123,
       0.99162011, 0.53910615, 0.82960894, 0.92458101, 0.94134078,
       0.97206704, 0.79329609, 0.82960894, 0.44413408, 0.93575419,
       0.98882682, 0.92178771, 0.86871508, 0.47486034, 0.98882682,
       0.98882682, 0.82960894, 0.86871508, 0.        , 0.95530726,
       0.98882682, 0.79329609, 0.86871508, 0.        , 0.95530726,
       0.98882682, 0.79329609, 0.86871508, 0.        , 0.95530

In [27]:
int2not2=dict((note2int[x],x) for x in note2int)

In [28]:
pred_out=[]

In [29]:
for ni in range(200):
    predi=np.reshape(pattern,(1,len(pattern),1))
    #print(predi)
    #predi=predi/len(note2int)
    
    prediction= model.predict(predi)
    i=np.argmax(prediction)
    res=int2not2[i]
    pred_out.append(res)
    
    pattern=np.append(pattern,i/len(note2int))
    pattern=pattern[1:]

In [31]:
print(len(pred_out))

200


In [32]:
pred_out

['A2',
 'A2',
 '2.5.9',
 'F2',
 'D2',
 'A2',
 'D3',
 'A2',
 'C5',
 'C5',
 'F2',
 'F2',
 'C3',
 'F2',
 'E4',
 '2.7',
 'G#4',
 'C5',
 'G2',
 'G4',
 'A4',
 'A2',
 '5.9',
 'E3',
 'D3',
 'F3',
 'E-3',
 'E-3',
 'E-3',
 'D5',
 'D5',
 'F3',
 'D5',
 'D5',
 'E5',
 'G4',
 'F#5',
 '1',
 'E5',
 'C5',
 'F4',
 'E4',
 'F4',
 'G4',
 'G#4',
 'D3',
 'G2',
 'B-3',
 'D6',
 'G2',
 'D5',
 'F3',
 'A4',
 'B-4',
 'F4',
 'G4',
 'E3',
 'A4',
 'D5',
 'D3',
 'A3',
 'A2',
 'D4',
 'D3',
 'A4',
 'F#3',
 '4.9',
 'D5',
 'A3',
 'E4',
 'F3',
 'B2',
 'F#3',
 'D3',
 'E3',
 'A3',
 '2.6.9',
 'B4',
 'F#4',
 'G3',
 'C3',
 'F3',
 'C#5',
 'A4',
 'C#5',
 'D5',
 'F3',
 '7',
 'G#2',
 'E3',
 'D5',
 'F#3',
 'B-3',
 'G#3',
 'B5',
 'B3',
 'E3',
 'D4',
 'A3',
 '2.8',
 'D6',
 'B-3',
 'G5',
 'B5',
 'C6',
 '0.4.7',
 'F2',
 'C#3',
 'C#3',
 'C#3',
 'C5',
 'C#3',
 'B4',
 'G#4',
 'F2',
 'F2',
 'F3',
 'B2',
 '11',
 'A3',
 'F#5',
 'G#4',
 'C4',
 'F#3',
 'G#4',
 'A4',
 'B4',
 'F#5',
 'C6',
 '9.0.4',
 'G#4',
 'B-3',
 'A3',
 'G#4',
 'B-3',
 'D4',
 '

In [33]:
offset = 0
output_notes = []

for pattern in pred_out:
    if(('.' in pattern) or pattern.isdigit()):
        
        print("Chord ->",pattern)
        notesinchord=pattern.split(".")
        notes=[]
        for cur in notesinchord:
            newnote=note.Note(int(cur))
            newnote.storedInstrument=instrument.Piano()
            notes.append(newnote)
        newchord= chord.Chord(notes)
        newchord.offset= offset
        output_notes.append(newchord)
    else:
        newnote=note.Note(pattern)
        newnote.offset = offset
        newnote.storedInstrument = instrument.Piano()
        output_notes.append(newnote)
    
    offset+=1.0
    
        
    
    

Chord -> 2.5.9
Chord -> 2.7
Chord -> 5.9
Chord -> 1
Chord -> 4.9
Chord -> 2.6.9
Chord -> 7
Chord -> 2.8
Chord -> 0.4.7
Chord -> 11
Chord -> 9.0.4
Chord -> 6.9
Chord -> 6.11
Chord -> 4.7.11
Chord -> 5
Chord -> 0
Chord -> 0
Chord -> 0
Chord -> 2
Chord -> 8
Chord -> 3.8


In [34]:
from music21 import stream

In [49]:
midistream= stream.Stream(output_notes)
midistream.write('midi',fp='C:/Users/user/predction.mid')

'C:/Users/user/predction.mid'

In [39]:
import music21
mf= music21.midi.translate.streamToMidiFile(midistream)

In [41]:
mf.

<MidiFile 1 tracks
  <MidiTrack 1 -- 874 events
    <MidiEvent DeltaTime, t=0, track=1, channel=1>
    <MidiEvent SEQUENCE_TRACK_NAME, t=0, track=1, channel=1, data=b''>
    <MidiEvent DeltaTime, t=0, track=1, channel=1>
    <MidiEvent PITCH_BEND, t=0, track=1, channel=1, parameter1=0, parameter2=64>
    <MidiEvent DeltaTime, t=0, track=1, channel=1>
    <MidiEvent NOTE_ON, t=0, track=1, channel=1, pitch=45, velocity=90>
    <MidiEvent DeltaTime, t=1024, track=1, channel=1>
    <MidiEvent NOTE_OFF, t=0, track=1, channel=1, pitch=45, velocity=0>
    <MidiEvent DeltaTime, t=0, track=1, channel=1>
    <MidiEvent NOTE_ON, t=0, track=1, channel=1, pitch=45, velocity=90>
    <MidiEvent DeltaTime, t=1024, track=1, channel=1>
    <MidiEvent NOTE_OFF, t=0, track=1, channel=1, pitch=45, velocity=0>
    <MidiEvent DeltaTime, t=0, track=1, channel=1>
    <MidiEvent NOTE_ON, t=0, track=1, channel=1, pitch=62, velocity=90>
    <MidiEvent DeltaTime, t=0, track=1, channel=1>
    <MidiEvent NOTE_ON, t=

In [45]:
note1= output_notes[0]
mstr= stream.Stream(note1)
mstr.write('midi',tp='example2.midi')

'C:\\Users\\user\\AppData\\Local\\Temp\\music21\\tmpb6oqgf0m.mid'

In [48]:
midistream.show('midi')